# Worksheet 08

Name:  Mark Maci
UID: U30478693

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [60]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.6533854052363814, 5.865902329720861, 6.7088502466361, 5.954423645755982, 3.954839916755966, 4.786852785499576, 6.152823862981239, 3.8969484354512103, 4.422111241677011, 6.037352670779008]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [61]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.065493957743201, 8.646606347756677, 8.294319869642875, 8.518425335189377, 8.10049406203236, 7.747124784657154, 8.564181319450386, 6.641888302502129, 6.601762298008909, 7.383515871902653]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [62]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.383515871902653, 6.601762298008909, 6.641888302502129, 8.564181319450386, 6.037352670779008, 4.422111241677011, 7.747124784657154, 8.10049406203236, 8.518425335189377, 8.294319869642875]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean, variance, mixing coefficient

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [63]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.383515871902653, 8.564181319450386, 7.747124784657154, 8.10049406203236, 8.518425335189377, 8.294319869642875]
[6.601762298008909, 6.641888302502129, 6.037352670779008, 4.422111241677011]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 8.101343540479135,  mean_2 = 5.925778628241765
var_1 = 0.1776940178504867,  var_2 = 0.8108078409369298


/Users/markmaci/miniconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [64]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x_i = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x_i)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x_i)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.383515871902653
probability of observing that point if it came from cluster 0 =  0.000642098103734708
probability of observing that point if it came from cluster 1 =  0.09774383845456291
point =  6.601762298008909
probability of observing that point if it came from cluster 0 =  7.696280082536315e-16
probability of observing that point if it came from cluster 1 =  0.3475816320472898
point =  6.641888302502129
probability of observing that point if it came from cluster 0 =  5.044695013698115e-15
probability of observing that point if it came from cluster 1 =  0.3331241543661933
point =  8.564181319450386
probability of observing that point if it came from cluster 0 =  0.07551299225443225
probability of observing that point if it came from cluster 1 =  0.0024698232275245004
point =  6.037352670779008
probability of observing that point if it came from cluster 0 =  1.1328267364812313e-29
probability of observing that point if it came from cluster 1 =  0.4873940294754382
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [70]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x)]
mean = [
    sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
    sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)
]
var = [
    sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
    sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)
]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.48883683522807375,  P(C_2) = 0.5111631647719261
mean_1 = 8.249131591826846,  mean_2 = 6.257567867361522
var_1 = 0.08841118628569777,  var_2 = 1.0418855187001184


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [71]:
prob_c0_x_updated = []
prob_c1_x_updated = []

for p in data:
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x_updated.append(pdf_i[0] * prob_c[0] / prob_x_i)
    prob_c1_x_updated.append(pdf_i[1] * prob_c[1] / prob_x_i)

probs = zip(data, prob_c0_x_updated, prob_c1_x_updated)

for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


7.383515871902653
Probability of coming from C_1 = 3.08929870238008e-20
Probability of coming from C_2 = 1.0

6.601762298008909
Probability of coming from C_1 = 4.832978219938376e-75
Probability of coming from C_2 = 1.0

6.641888302502129
Probability of coming from C_1 = 2.0799084736790647e-71
Probability of coming from C_2 = 1.0

8.564181319450386
Probability of coming from C_1 = 0.1859787288106438
Probability of coming from C_2 = 0.8140212711893561

6.037352670779008
Probability of coming from C_1 = 1.446812983378739e-135
Probability of coming from C_2 = 1.0

4.422111241677011
Probability of coming from C_1 = 0.0
Probability of coming from C_2 = 1.0

7.747124784657154
Probability of coming from C_1 = 3.1244516595702132e-06
Probability of coming from C_2 = 0.9999968755483404

8.10049406203236
Probability of coming from C_1 = 0.9291228513361754
Probability of coming from C_2 = 0.07087714866382454

8.518425335189377
Probability of coming from C_1 = 0.5343661278181066
Probability of comi

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [73]:
assignments = []

for i in range(len(data)):
    if prob_c0_x_updated[i] > prob_c1_x_updated[i]:
        assignments.append(0)
    else:
        assignments.append(1)

for i, point in enumerate(data):
    print(f"Point {point} is assigned to cluster {assignments[i]}")
    print()


Point 7.383515871902653 is assigned to cluster 1

Point 6.601762298008909 is assigned to cluster 1

Point 6.641888302502129 is assigned to cluster 1

Point 8.564181319450386 is assigned to cluster 1

Point 6.037352670779008 is assigned to cluster 1

Point 4.422111241677011 is assigned to cluster 1

Point 7.747124784657154 is assigned to cluster 1

Point 8.10049406203236 is assigned to cluster 0

Point 8.518425335189377 is assigned to cluster 0

Point 8.294319869642875 is assigned to cluster 0



### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Yes      |
| A  C |      No    |
| A  D |      No      |
| A  E |      No       |
| B  C |      No       |
| B  D |      No     |
| B  E |      No       |
| C  D |      Yes       |
| C  E |     Yes     |
| D  E |     No     |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

The formula for the number of unique pairs of points one can create is N choose 2, or N(N-1)/2.

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

Maximum number of disagreements = 9 choose 2 = 36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Total agreements = 10
Total disagreements = 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Given clusterings C and P, calculate the number of agreements and disagreements for each cluster. Then, sum the number of agreements and disagreements across all clusters. The total number of disagreements is the sum of the number of disagreements for each cluster.